# Phase 1: Jigsaw Puzzle Preprocessing Pipeline

This notebook implements the full classical computer vision pipeline for preprocessing jigsaw puzzle images according to the project requirements. It includes:

- Color normalization
- Denoising
- Edge enhancement
- Foreground segmentation
- Contour extraction
- Cropping of individual pieces
- Saving all artifacts for Milestone 2

In [ ]:
import os
import cv2
import numpy as np
import json
from pathlib import Path
from typing import List, Tuple, Dict
import matplotlib.pyplot as plt

def imshow(img, cmap='gray', title='', size=8):
    plt.figure(figsize=(size, size))
    if img.ndim == 2:
        plt.imshow(img, cmap=cmap)
    else:
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.title(title)
    plt.axis('off')
    plt.show()


In [ ]:
INPUT_IMAGE = "dataset_images/sample.jpg"  # Replace with your actual image path
OUTPUT_DIR = "phase1_outputs"
os.makedirs(OUTPUT_DIR, exist_ok=True)


In [ ]:
def color_normalize_clahe(img_bgr):
    lab = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    l2 = clahe.apply(l)
    lab2 = cv2.merge((l2, a, b))
    return cv2.cvtColor(lab2, cv2.COLOR_LAB2BGR)

def denoise_image(img_bgr):
    return cv2.bilateralFilter(img_bgr, 9, 75, 75)

def edge_enhance(img_bgr):
    img_gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)
    gaussian = cv2.GaussianBlur(img_gray, (0, 0), sigmaX=3)
    unsharp = cv2.addWeighted(img_gray, 1.5, gaussian, -0.5, 0)
    return cv2.cvtColor(unsharp, cv2.COLOR_GRAY2BGR)


In [ ]:
def compute_mask(img_bgr):
    gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)
    th = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                               cv2.THRESH_BINARY_INV, 51, 8)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (9, 9))
    closed = cv2.morphologyEx(th, cv2.MORPH_CLOSE, kernel, iterations=2)
    opened = cv2.morphologyEx(closed, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5,5)), iterations=1)
    num_labels, labels, stats, _ = cv2.connectedComponentsWithStats(opened, connectivity=8)
    mask = np.zeros_like(opened)
    for i in range(1, num_labels):
        if stats[i, cv2.CC_STAT_AREA] >= 1000:
            mask[labels == i] = 255
    return mask

def extract_contours(mask):
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    return [c for c in contours if cv2.contourArea(c) >= 1000]


In [ ]:
img = cv2.imread(INPUT_IMAGE)
imshow(img, title="Original")

img_clahe = color_normalize_clahe(img)
imshow(img_clahe, title="CLAHE Normalized")

img_denoised = denoise_image(img_clahe)
imshow(img_denoised, title="Denoised")

img_sharp = edge_enhance(img_denoised)
imshow(img_sharp, title="Edge Enhanced")

mask = compute_mask(img_denoised)
imshow(mask, title="Mask")

contours = extract_contours(mask)
img_contour = img.copy()
cv2.drawContours(img_contour, contours, -1, (0,255,0), 2)
imshow(img_contour, title="Contours")


In [ ]:
def save_cropped_pieces(img, contours, out_dir, prefix="piece"):
    os.makedirs(out_dir, exist_ok=True)
    for i, cnt in enumerate(contours):
        x, y, w, h = cv2.boundingRect(cnt)
        pad = int(0.05 * max(w, h))
        x0 = max(0, x - pad)
        y0 = max(0, y - pad)
        x1 = min(img.shape[1], x + w + pad)
        y1 = min(img.shape[0], y + h + pad)
        crop = img[y0:y1, x0:x1].copy()
        filename = os.path.join(out_dir, f"{prefix}_{i:02d}.png")
        cv2.imwrite(filename, crop)

save_cropped_pieces(img, contours, os.path.join(OUTPUT_DIR, "pieces"))
print(f"Saved {len(contours)} pieces to {OUTPUT_DIR}/pieces/")
